### Finding analytical solution

In [1]:
from sympy import *
import numpy as np

In [2]:
# Define parameters in atomic units
hbar = 1
w0 = 5  # atom transition frequency 
wL = 5  # laser frequency
det = wL - w0  # detuning from resonance
d = 1  # transition dipole moment
E = 0.2  # electric field amplitude
b = 0.05 # spontaneous emission coefficient (gamma)
times = np.linspace(0, 100, 1001)  # creates list of 10001 points evenly distributed between values 0 and 100

a = E * d / hbar # Rabi frequency (omega_R)
c = np.sqrt(a**2 - b**2/16) # uppercase gamma

In [3]:
#Define and solve the linear equations
#omega_R = a, gamma = b, uppercase gamma = c
c_1, c_2, c_3, c_4 = symbols('c_1 c_2 c_3 c_4')
a,b,c = symbols('a b c', real = True)

eq1 = Eq(c_1 - 4*(a**2)/(b**2 + 8*(a**2)), 0)
eq2 = Eq(c_1 + c_2 + c_3 + c_4, 0)
eq3 = Eq(-c_2 + (-3/2 + 2j*c/b)*c_3 + (-3/2 - 2j*c/b)*c_4, 0)
eq4 = Eq((b**2/4)*c_2 + ((3*b/4 - 1j*c)**2)*c_3 + ((3*b/4 + 1j*c)**2)*c_4, 2*(a**2))
sol = solve((eq1, eq2, eq3, eq4),(c_1, c_2, c_3, c_4))
sol

{c_1: 4.0*a**2/(8.0*a**2 + b**2),
 c_2: (256.0*a**4 - 4.0*a**2*b**2 - 64.0*a**2*c**2)/(8.0*a**2*b**2 + 128.0*a**2*c**2 + b**4 + 16.0*b**2*c**2),
 c_3: (32.0*I*a**4 + I*a**2*b**2 + 4.0*a**2*b*c)/(8.0*a**2*b*c - 32.0*I*a**2*c**2 + b**3*c - 4.0*I*b**2*c**2),
 c_4: (-32.0*I*a**4 - I*a**2*b**2 + 4.0*a**2*b*c)/(8.0*a**2*b*c + 32.0*I*a**2*c**2 + b**3*c + 4.0*I*b**2*c**2)}

In [4]:
f = sol[c_2]/sol[c_1]

g = sol[c_3]/sol[c_1]
h = sol[c_4]/sol[c_1]

#simplify(f)
simplify(g)
#simplify(h)

c_2_1 = simplify(f)
c_3_1 = simplify(g)
c_4_1 = simplify(h)

simplify(g)


0.25*(8.0*a**2 + b**2)*(32.0*I*a**2 + I*b**2 + 4.0*b*c)/(c*(8.0*a**2*b - 32.0*I*a**2*c + b**3 - 4.0*I*b**2*c))

In [5]:
#Simplify expression for rho_ee element of density matrix
t = symbols('t')
'''def rho_ee(t):
    rho_ee = simplify(sol[c_1] + exp(-b*t/2)*sol[c_2] + exp((-3*b/4 + 1j*c)*t)*sol[c_3] + exp((-3*b/4 - 1j*c)*t)*sol[c_4])
    return rho_ee
rho_ee_new = rho_ee(times)'''

rho_ee = simplify(sol[c_1] + exp(-b*t/2)*sol[c_2] + exp((-3*b/4 + 1j*c)*t)*sol[c_3] + exp((-3*b/4 - 1j*c)*t)*sol[c_4])
rho_ee

rho_ee_1 = sol[c_1]*simplify(1 + exp(-b*t/2)*c_2_1 + exp((-3*b/4 + 1j*c)*t)*c_3_1 + exp((-3*b/4 - 1j*c)*t)*c_4_1)
rho_ee_1

simplify(sqrt(c_3_1*conjugate(c_3_1)))

sqrt(1/(c**2*(64.0*a**4*b**2 + 1024.0*a**4*c**2 + 16.0*a**2*b**4 + 256.0*a**2*b**2*c**2 + b**6 + 16.0*b**4*c**2)))*(64.0*a**2 + 8.0*b**2)*sqrt(a**4 + 0.0625*a**2*b**2 + 0.0009765625*b**4 + 0.015625*b**2*c**2)

In [68]:
conjugate(c_3_1)

0.25*(8.0*a**2 + b**2)*(-32.0*I*a**2 - I*b**2 + 4.0*b*c)/(c*(8.0*a**2*b + 32.0*I*a**2*c + b**3 + 4.0*I*b**2*c))

### Finding numerical solution

In [ ]:
from qutip import *

In [ ]:
e = basis(2, 0)  # excited state
g = basis(2, 1)  # ground state

In [ ]:
rho0 = g * g.dag()  # initially system in the ground state

In [ ]:
H0 = 0.5 * hbar * w0 * sigmaz()  # time-independent part
H1 = [-0.5 * E * d * sigmax(), "exp(1j*wL*t) + exp(-1j*wL*t)"]  # time-dependent part as a list where second term is f(t)
H = [H0, H1]  # the whole Hamiltonian has to be write as a list of all parts

In [ ]:
rho = mesolve(H, rho0, times, [np.sqrt(y)*sigmam()], [], args = {"wL": wL}, progress_bar=None)

### Plotting numerical and analytical solutions

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
fig, ax = plt.subplots(1, figsize=(15, 8)) 
ax.plot(times, expect(sigmaz(), rho.states), linewidth=2, label='full')  # plotting full <sigma_z>
ax.plot(times, 2 * rho_ee_1 - 1, '--', linewidth=2, label='analytical')  # plotting analytical <sigma_z>
ax.set_xlabel('times')  # set label of x-axis
ax.set_ylabel(r'Tr($\rho\cdot\sigma_z$)')  # set label of y-axes (LateX formulas are used here)
ax.grid()  # plot grid
plt.legend()
plt.show()  # show the results